# longseq

longer dataset을 사용하고 싶음.

we want to train in bigger chunks

In [1]:
import torch
import torch.optim as optim
import numpy as np

# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

X->Y

"if you wan" -> "f you want" 

"f you want" -> " you want "

이런식으로..


In [5]:
# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

#data setting
x_data=[]
y_data=[]

for i in range(0,len(sentence)-sequence_length):            #우리가 가져올 수 있는 chunk의 개수
    x_str=sentence[i:i+sequence_length]
    y_str=sentence[i+1:i+1+sequence_length]
    print(i, x_str, y_str)
    x_data.append([char_dic[c] for c in x_str])             #x_str to index
    y_data.append([char_dic[c] for c in y_str])             #y_str to index

x_one_hot=[np.eye(dic_size)[x] for x in x_data]

X=torch.FloatTensor(x_one_hot)
Y=torch.LongTensor(y_data)

0 if you wan f you want
1 f you want  you want 
2  you want  you want t
3 you want t ou want to
4 ou want to u want to 
5 u want to   want to b
6  want to b want to bu
7 want to bu ant to bui
8 ant to bui nt to buil
9 nt to buil t to build
10 t to build  to build 
11  to build  to build a
12 to build a o build a 
13 o build a   build a s
14  build a s build a sh
15 build a sh uild a shi
16 uild a shi ild a ship
17 ild a ship ld a ship,
18 ld a ship, d a ship, 
19 d a ship,   a ship, d
20  a ship, d a ship, do
21 a ship, do  ship, don
22  ship, don ship, don'
23 ship, don' hip, don't
24 hip, don't ip, don't 
25 ip, don't  p, don't d
26 p, don't d , don't dr
27 , don't dr  don't dru
28  don't dru don't drum
29 don't drum on't drum 
30 on't drum  n't drum u
31 n't drum u 't drum up
32 't drum up t drum up 
33 t drum up   drum up p
34  drum up p drum up pe
35 drum up pe rum up peo
36 rum up peo um up peop
37 um up peop m up peopl
38 m up peopl  up people
39  up people up people 
40 up peop

C:\Users\dhlee\AppData\Local\Temp\ipykernel_23284\4210097221.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  X=torch.FloatTensor(x_one_hot)


# 기본적인 fully connected layer와 RNN



In [ ]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net,self).__init__()
        self.rnn=torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc=torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self,x):
        x,_status=self.rnn(x)
        x=self.fc(x)
        return x

net=Net(dic_size, hidden_size, 2)               # num_layers가 2개 라는 소리.

NameError: name 'hidden_dim' is not defined

In [15]:
import torch
import torch.optim as optim
import numpy as np

# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

# make dictionary
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}

# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

# data setting
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

class NET(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(NET, self).__init__()
        self.rnn=torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc=torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
    
    def forward(self, x):
        x,_status=self.rnn(x)
        x=self.fc(x)
        return x
    
net=NET(dic_size, hidden_size,2)
    
criterion=torch.nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(), learning_rate)

for i in range(100):
    optimizer.zero_grad()
    outputs=net(X)
    loss=criterion(outputs.view(-1,dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results=outputs.argmax(dim=2)                           # results는 모든 시퀀스에 대해 각 시점에서 가장 확률이 높은 클래스(문자)의 인덱스를 포함한 배열
    predict_str=""
    for j, result in enumerate(results):
        print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j==0:
            predict_str+=''.join([char_set[t] for t in result])
        else:
            predict_str+=char_set[result[-1]]               # result[-1]은 가장 마지막의 녀석을 가지고 오라는 소리다. 


0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [11]:
#참고
print(X.shape)
print(Y.shape)

torch.Size([170, 10, 25])
torch.Size([170, 10])
